# Reading/Writing Images in Python

*Dr Chas Nelson*

*Part of https://github.com/ChasNelson1990/image-processing-in-python*

## Objectives

* Be able to read multidimensional image stacks, primarily multi-page TIFF files
* Be able to write multidimensional NumPy arrays to multi-page TIFF files

## Data

The image we will use for the rest of this tutorial is from the Broad Bioimage Benchmark Collection data set BBBC0034v1 (https://data.broadinstitute.org/bbbc/; Thirstrup et al. 2018).

See https://data.broadinstitute.org/bbbc/BBBC034/ for the full description; however, the key points are:

* $1024 \times 1024 \times 52$ pixels
* $65 \times 65 \times 290$ nm/pixel
* 4 channels (each stored as separate files):
  * Cell membrane label (C=0)
  * Actin label (C=1)
  * DNA label (C=2)
  * Brightfield image (C=3)

In [ ]:
# Run this cell to download and unzip the images we need
# You do not need to understand it, but happy to explain during coffee
# You only need to run this cell once and then you may comment it out
import urllib.request
import io
import zipfile
import os
from tqdm import tqdm_notebook, tnrange

url = 'https://data.broadinstitute.org/bbbc/BBBC034/BBBC034_v1_dataset.zip'

with urllib.request.urlopen(url) as response:
    print("Downloading...")
    length = int(response.getheader('content-length'))
    chunk = max(4096, length//9999)
    
    buffer = io.BytesIO()
    size = 0
    for b in tnrange(length//chunk + 1):
        block = response.read(chunk)
        if not block:
            print("Finished reading after {0}% of file.".format(size/length))
        buffer.write(block)
        size = size + len(block)
    print("Finished reading file.")
    
    print("Unzipping... ",end="")
    zf = zipfile.ZipFile(buffer)
    os.makedirs('./assets/bbbc034v1/',exist_ok=True)
    zf.extractall(path='./assets/bbbc034v1')
    print("Complete.")

## Reading Multidimensional TIF files

We will use the [`tifffile` plug-in](https://www.lfd.uci.edu/~gohlke/code/tifffile.py.html) for reading and writing multidimensional TIF files. This is includedas part of the [`io` submodule](https://scikit-image.org/docs/stable/api/skimage.io.html) within the popular image processing module [`scikit-image`](https://scikit-image.org).

The `imread` function is able to deal with any multidimensional TIF file that `tiffile` is capable of reading - this means pretty much any TIF that's compatabile with ImageJ/FIJI.

### Aside: Reading 'Collections'

* A 'collection' is a series of files containing a single 2D image whose names follow a name pattern, e.g. `file1.tif`, `file2.tif`, `file3.tif` all follow the pattern `file*.tif`.
* Collections may represent a time series, where each individual file could be a 2D TIF file, or a z-stack.
* One benefit with collections is that `scikit-image` can load frames into memory only when needed.
* Collections can read using, for example,

```python
from skimage import io

collection = io.imread_collection('./file*.tif',conserve_memory=True)
frame1 = collection[0]
```

In [ ]:
from skimage import io

# Read a single multidimensional TIF file, in this case a single channel with multiple z-slices.
c1 = io.imread('./assets/bbbc034v1/AICS_12_134_C=1.tif')

# Confirm that this image is in fact a numpy array
display(type(c1))

# Metadata for future use later
x_pixel_size = 65  # nm
y_pixel_size = 65  # nm
z_pixel_size = 290  # nm

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> In the cell below confirm the size of these images in pixels and the bit-depth of the image.</div>

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task</strong> Using the codes above, create a new code cell and make a figure showing maximum projections in all three dimensions.</div>

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task</strong> Using the codes above, create a new code cell and make an interactive figure allowing you to scroll through $z$.</div>

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task</strong> Using the information above, read the other three channels for this data into the variables <code>c0</code>, <code>c2</code> and <code>c3</code>.</div>

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task</strong> Use your existing Python skills to combine these four arrays into a single, 4D NumPy array called <code>my4DArray</code>. Print its shape to confirm your dimensions are in the right order.</div>

## Saving Multidimensional TIF files

Like `imread`, the `scitkit-image` function `imsave` is able to deal with any multidimensional TIF file that `tiffile` is capable of saving - again, this means pretty much any TIF file that's compatabile with ImageJ/FIJI.

We can also include metadata in our TIF file, which ImageJ/FIJI can then read and use as appropriate.

In [ ]:
# Save our multicolour stack to a new ImageJ-compatible TIF file
# tifffile wants our array to be in TZCYXS order for imageJ compatibility
my4DArray = np.moveaxis(my4DArray,-1,1)  # so move C from the end to second dimension
io.imsave('./assets/bbbc034v1/AICS_12_134_C=all.tif',
          my4DArray,
          imagej=True,
          resolution=(1/x_pixel_size,1/y_pixel_size),
          metadata={'spacing':z_pixel_size,'unit':'nm'})

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task</strong> Open your new file in ImageJ/FIJI and check that the dimensions are correct and that the metadata has been read.</div>

## Key Points

* We can use `scikit-image` to read and write multidimensional TIF files
* Multidimensional image stacks, e.g. multi-page TIFF files, can be read into Python and NumPy arrays with `imread`
* Multidimensional NumPy arrays can be saved as image files, e.g. ImageJ-compatible TIFF files with `imwrite`

## Any Bugs/Issues/Comments?

If you've found a bug or have any comments about this notebook, please fill out this on-line form:

Any feedback I get I will try to correct/implement as soon as possible.